# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    # exp_return(data, 40) should output 0.07500000000000002
    
    expected_return = 0.0
    
    for i in range(len(data)):
        possible_return = (data['Price in 1 Year'][i] / P) - 1
        expected_return += data['Probability'][i] * possible_return
    
    return expected_return

#exp_return(data, 40)

Write the function ```volatility```, which calculates the volatility of the stock, given the current stock price ```P```.

In [4]:
def volatility(data, P):
    # volatility(data,40) should output 0.21301408404140795

    expected_return = exp_return(data, P)
    variance = 0.0

    for i in range(len(data)):
        possible_return = (data['Price in 1 Year'][i] / P) - 1
        variance += data['Probability'][i] * (possible_return ** 2)

    variance -= expected_return ** 2
    standard_deviation = np.sqrt(variance)
    
    return standard_deviation

#volatility(data, 40)

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [5]:
def interval(data, P):
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    
    expected_return = exp_return(data, P)
    SD = volatility(data, P)
    SE = SD / np.sqrt(len(data))
    
    lower_bound = expected_return - (1.96 * SE)
    upper_bound = expected_return + (1.96 * SE)
    
    return [lower_bound, upper_bound]

#interval(data, 40)

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [6]:
def exp_return_capm(beta, market_return, rf):
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    
    expected_return_capm = rf + (beta * (market_return - rf))
    return expected_return_capm

#exp_return_capm(-0.6, 0.1, 0.02)

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [7]:
def clean_data(path):
    # This should yield data with 403316 rows × 4 columns
    
    # Read the CSV file
    data = pd.read_csv(path)

    # Remove observations with a non-numeric RET
    data = data[pd.to_numeric(data['RET'], errors='coerce').notna()]

    # Count the number of observations for each security
    security_counts = data['PERMNO'].value_counts()

    # Filter out securities with less than 500 observations
    securities_to_keep = security_counts[security_counts >= 500].index
    data = data[data['PERMNO'].isin(securities_to_keep)]

    # Keep only the specified columns
    data = data[['PERMNO', 'date', 'CUSIP', 'RET']]

    return data

#clean_data('monthlycrsp_mod6.csv')

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [8]:
def avg_return(data):
    # This should yield data with 679 rows × 2 columns
    
    # Convert the 'RET' column to a numeric data type and set non-numeric values to NaN
    data['RET'] = pd.to_numeric(data['RET'], errors='coerce')
    
    # Group by CUSIP and calculate the average return for each security
    avg_returns = data.groupby('CUSIP')['RET'].mean().reset_index()
    avg_returns.columns = ['CUSIP', 'Average Return']
    
    return avg_returns
    
#avg_return(clean_data('monthlycrsp_mod6.csv'))

In [9]:
def security_volatilities(data):
    # This should yield data with 679 rows × 2 columns
    
    # Convert the 'RET' column to a numeric data type and set non-numeric values to NaN
    data['RET'] = pd.to_numeric(data['RET'], errors='coerce')
    
    # Group by CUSIP and calculate the standard deviation (volatility) of returns for each security
    volatilities = data.groupby('CUSIP')['RET'].std().reset_index()
    volatilities.columns = ['CUSIP', 'Volatility']
    
    return volatilities
    
#security_volatilities(clean_data('monthlycrsp_mod6.csv'))

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [10]:
def correlation(ret_data, vol_data):
    # correlation(rets, vols) should yield 0.44837090727059614
    
    # Merge the ret_data and vol_data DataFrames on 'CUSIP' to calculate the correlation
    merged_data = pd.merge(ret_data, vol_data, on='CUSIP')
    
    # Calculate the correlation between 'Average Return' and 'Volatility'
    corr = merged_data['Average Return'].corr(merged_data['Volatility'])
    
    return corr

#correlation(avg_return(clean_data('monthlycrsp_mod6.csv')), security_volatilities(clean_data('monthlycrsp_mod6.csv')))